In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pathlib

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

training_dir = pathlib.Path('/kaggle/input/radar-signal-classification/training_set')

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

full_dataset = datasets.ImageFolder(root=str(training_dir), transform=transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

class_names = full_dataset.classes  
num_classes = len(class_names)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Classes: {class_names}")
print(f"Training samples: {train_size}, Validation samples: {val_size}")


class SimpleCNN(nn.Module):
    def __init__(self, num_classes=8):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.batchnorm3 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(2, 2)  
        
        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(F.relu(self.batchnorm1(self.conv1(x))))  
        x = self.pool(F.relu(self.batchnorm2(self.conv2(x))))  
        x = self.pool(F.relu(self.batchnorm3(self.conv3(x))))  

        x = x.view(x.size(0), -1)  
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)


model = SimpleCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    train_loss, val_loss, train_acc, val_acc = [], [], [], []
    
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        train_loss.append(running_loss / len(train_loader))
        train_acc.append(correct / total)

        model.eval()
        val_running_loss, correct, total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_loss.append(val_running_loss / len(val_loader))
        val_acc.append(correct / total)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {train_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f} - "
              f"Val Loss: {val_loss[-1]:.4f}, Val Acc: {val_acc[-1]:.4f}")

    return model


epochs = 5
model = train(model, train_loader, val_loader, criterion, optimizer, epochs)
example_input = torch.randn(1, 3, 128, 128).to(device)  
traced_model = torch.jit.trace(model, example_input)

# sinh vien luu model voi name la mssv
traced_model.save("22111001.pt")
print("Model saved as 22111001.pt")


Classes: ['B-FM', 'Barker', 'CPFSK', 'DSB-AM', 'GFSK', 'LFM', 'Rect', 'SSB-AM']
Training samples: 5120, Validation samples: 1280
Epoch 1/5 - Train Loss: 1.5230, Train Acc: 0.5594 - Val Loss: 0.5625, Val Acc: 0.7742
Epoch 2/5 - Train Loss: 0.6030, Train Acc: 0.7324 - Val Loss: 0.4669, Val Acc: 0.7961
Epoch 3/5 - Train Loss: 0.5175, Train Acc: 0.7533 - Val Loss: 0.5442, Val Acc: 0.7211
Epoch 4/5 - Train Loss: 0.4552, Train Acc: 0.7824 - Val Loss: 0.3687, Val Acc: 0.8102
Epoch 5/5 - Train Loss: 0.4014, Train Acc: 0.8094 - Val Loss: 0.3726, Val Acc: 0.8211
Model saved as 22111001.pt
